# Operation of Running Systems
- Boot Process
  - The Boot Process on an x86 system can be categorized into 4 sequential phases:
    - The Firmware phase
      - BIOS runs the POST
      - Video drivers are installed
      - The BIOS scans for a storage device containing a boot loader. This device will contain a master boot record. 
        A Master Boot Record contains a 512 byte image containing 446 bytes of the bootloader program, 64 bytes for the 
        partition table, and the last 2 bytes for the boot signature. 
      - The MBR is located on the first sector of the boot disk.
    - The Bootloader Phase
      - GRUB2 is a common boot loader used in Linux
      - The primary job of the boot loader is to locate the linux kernel (in /boot), decompress it, load it into memory based
        on the configuration in `/etc/grub2/grub.cfg`, and pass control to it. 
      - For EFI Systems, grub2 loads the kernel based on the `/boot/efi/EFI/centos/grub.efi` file
      - The `grub.cfg` and `grub.efi` files are generated automatically based on the `/etc/default/grub2` file. These files are regenerated
        automatically when a new kernel is installed, so you should not modify them directly. Instead, make any changes to `/etc/default/grub2`
      - After modifying `/etc/default/grub2`, you'll need to run `grub2-mkconfig` to regenerate the files in `/boot`
      - `grub2-mkconfig` uses scripts in `/etc/grub.d/` to help regenerate the grub2.cfg file
    - The Kernel Phase
      - After getting control from the bootloader, the kernel extracts the initial ramdisk (`/boot/initramfs`) file system image, decompresses
        it, and mounts it as read-only at `/sysroot`, from there it loads drivers and then mounts the root file system on `/` as read/write.
      - The kernel then executes the init daemon (systemd) with PID 1 and passes control to it
      - The kernel and its support files are spread across the system. Typically in `/boot`, `/proc`, and `/usr/lib/modules`
      - Always install new versions of the kernel instead of upgrading. The upgrade process removes the old kernel. In case of a problem,
        you will not be able to revert the upgrade.
    - The Init Phase
      - systemd starts all userspace system and network services and brings the system up to the preset boot target.


- Reboot/Shutdown
  - You can use `sudo systemctl reboot --force` to force a system to reboot
  - only the superuser can reboot 
  - `shutdown` takes a time parameter for scheduling a shutdown. 
    - ex: `shutdown 03:00` = shutdown at 3AM
    - `shutdown +15` shutdown in 15 minutes
- Operating Modes
  - `systemctl get-default` will show you the default target
  - Examples of operating modes:
    - graphical
    - multi-user
    - single-user
  - use `systemctl set-default multi-user.target` to set the default operating mode, then reboot
  - You can switch to a target without booting by typing `systemctl isolate graphical.target`, but this does not change the default boot target
  - Useful targets:
    - emergency.target = root file system is read-only. Minimal amount of programs loaded
    - rescue.target = a few services are loaded and you are dropped into a root shell
      - You must have a password set for the root user to use either of these operating modes
- Boot Loaders
  - The boot loader starts the linux kernel
  - GRUB is a popular boot loader for Linux
  - To generate a grub configuration file:
    - grub2-mkconfig -o /boot/grub2/grub.cfg for BIOS systems
    - grub2-mkconfig -o /boot/efi/EFI/grub.cfg for EFI systems
    - grub2-install can be used to install grub on a disk
    - The boot loader is typically stored on the first few sectors of a disk 
    - The grub2.cfg file is typically stored at /boot/grub2/grub.cfg for BIOS systems
  - /etc/default/grub is used by the grub2-mkconfig utility to determine what settings to use when it generates the grub2.cfg file. After you modify this file, you need to run `grub2-mkconfig` to actually regenerate the grub2 config
      ~~~shell
      GRUB_TIMEOUT=1
      GRUB_TIMEOUT_STYLE=countdown
      GRUB_DISTRIBUTOR="$(sed 's, release .*$,,g' /etc/system-release)"
      GRUB_DEFAULT=saved
      GRUB_DISABLE_SUBMENU=true
      GRUB_TERMINAL="serial console"
      GRUB_CMDLINE_LINUX="console=tty1 console=ttyS0,115200n8 earlyprintk=ttyS0,115200 rootdelay=300 scsi_mod.use_blk_mq=y crashkernel=auto"
      GRUB_DISABLE_RECOVERY="true"
      GRUB_ENABLE_BLSCFG=true
      GRUB_SERIAL_COMMAND="serial --speed=115200 --unit=0 --word=8 --parity=no --stop=1"
      ~~~
  - Install, Configure, and Troubleshoot BootLoaders
    - To regenerate grub2 config:
      - Boot into recovery media, then:
        - `chroot /mnt/sysroot`
        - To regenerate grub config for BIOS system
          - `grub2-mkconfig -o /boot/grub2/grub.cfg`
        - To regenerate grub config for EFI system:
          - `grub2-mkconfig -o /boot/efi/EFI/centos/grub.cfg`
    - To reinstall the boot loader:
      - BIOS systems:
        -  Use `lsblk` to look at block devices. Try to identify the boot device
        -  Use `grub2-install /dev/sda` to install grub to the boot device
     -  EFI Systems:
        -  Use `dnf reinstall grub2-efi grub2-efi-modules shim` to reinstall grub to the boot device

- Scripting
  - type `help` to get a list of bash-builtins
  - Some examples of bash built-ins:
    - `cd`, `pwd`, `source`, `exec`, `break`, `umask`, etc
    - Bash built-ins do not launch a new process
    - Commands in directories such as `/usr/bin` and `/usr/sbin` are external to the shell, and the
    shell launches a new sub-shell process for them
  - There are 2 types of unit configuration files:
    - system unit files are preinstalled with the operating system and are located at `/usr/lib/systemd/system`
    - User init files that are user defined and are located at `/etc/systemd/user`
      

- Manage startup processes and services
  - Systemd
    - The init system is responsible for starting the system and all unit files 
      - Units are systemd objecfts used for organizing boot and maintenance tasks. Units consist of mounts, services, sockets, devices, and timers, etc.
      There are 11 unit types
      - Services units tells the init system what it needs to know about the life cycle of an application
      - systemd is the init system typically
      - Use `systemctl cat sshd.service` to view the unit file for a service
      - Use `systemctl edit --full sshd.service` to edit a unit file
      - use `systemctl revert sshd.service` to revert the unit file to the default
    - To prevent a service from being started, you can mask it
      - systemctl mask sshd.service
    - Targets are simply logical collections of units. 
      - Target files end in the `.target` extension.
      - Systemd includes several predefined targets:
        - halt: shuts down and halts the system
        - poweroff: shuts down and powers off the system
        - shutdown: shuts down the system
        - rescue: boots into single user mode for recovery. All local file systems are mounted. Networking is disabled. Some essential services are started
        - emergency: Runs an emergency shell. The root file system is mounted in read-only mode, other file systems are not mounted. Network and other services are disabled
        - multi-user: full network support, but without a GUI
        - graphical: full network support with a GUI
        - reboot: shuts down and reboots the system
        - default: a special soft link that points to the default system boot target (multi-user or graphical)
        - hibernate: Puts the system into hibernation 

- Managing and Diagnosing Processes
  - At any time, a process can be in 1 of 5 states
    - Running: the process is being executed by the system
    - sleeping: the process is waiting for input from another process or user
    - waiting: the process has received any input is was waiting on and is now waiting to be scheduled on a CPU
    - stopped: The process is currently stopped and not running
    - zombie: the process is dead. The process still exists in the process table, but it takes up no system resources. 

  - When viewing output of `ps aux`, commands wrapped in square brackets are kernel level processes
    ```
    [azureadmin@centos01 ~]$ ps aux
    USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
    root           1  0.0  0.0 175236 13720 ?        Ss   Aug30   0:10 /usr/lib/systemd/systemd --switched-root --system --deserialize 17
    root           2  0.0  0.0      0     0 ?        S    Aug30   0:00 [kthreadd]
    root           3  0.0  0.0      0     0 ?        I<   Aug30   0:00 [rcu_gp]
    root           4  0.0  0.0      0     0 ?        I<   Aug30   0:00 [rcu_par_gp]
    root           6  0.0  0.0      0     0 ?        I<   Aug30   0:00 [kworker/0:0H-events_highpri]
    root           9  0.0  0.0      0     0 ?        I<   Aug30   0:00 [mm_percpu_wq]
    root          10  0.0  0.0      0     0 ?        S    Aug30   0:00 [ksoftirqd/0]
    root          11  0.0  0.0      0     0 ?        I    Aug30   0:32 [rcu_sched]
    root          12  0.0  0.0      0     0 ?        S    Aug30   0:00 [migration/0]
    root          13  0.0  0.0      0     0 ?        S    Aug30   0:00 [watchdog/0]
    root          14  0.0  0.0      0     0 ?        S    Aug30   0:00 [cpuhp/0]
    ```
  - Show all processes running as a user:
    ```
    [azureadmin@centos01 ~]$ ps -uU azureadmin
    USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
    azuread+    6236  0.0  0.0  89420  9372 ?        Ss   Aug30   0:01 /usr/lib/systemd/systemd --user
    azuread+    6239  0.0  0.0 253032  5064 ?        S    Aug30   0:00 (sd-pam)
    azuread+    6246  0.0  0.0 163788  5548 ?        R    Aug30   0:00 sshd: azureadmin@pts/0
    azuread+    6247  0.0  0.0 242252  5608 pts/0    Ss   Aug30   0:00 -bash
    azuread+   83085  0.0  0.0 278612  4020 pts/0    R+   08:44   0:00 ps -uU azureadmin
    ```
  - `pgrep` can be used to search for processes by name
    ```
    [azureadmin@centos01 ~]$ pgrep -a http
    941 /usr/sbin/httpd -DFOREGROUND
    974 /usr/sbin/httpd -DFOREGROUND
    975 /usr/sbin/httpd -DFOREGROUND
    976 /usr/sbin/httpd -DFOREGROUND
    977 /usr/sbin/httpd -DFOREGROUND
    1728 /usr/sbin/httpd -DFOREGROUND
    ```
  - niceness
    - Process niceness determines how nice a process is to other processes
    - can be -20 and +19
    - Lower the number, the less nice the process is to other processes
    - If ProcessB has a nice value of -20 and ProcessA has a nice value of +19, and both are competing for processor cycles, ProcessB will have priority
      - `nice -n 11 /bin/bash`
    - use `ps -l` or `ps -lax` to view the niceness of a process
    - Processes inherit the niceness value of their parent processes
    - a regular user can only set nice values between 0 and 19, anything below 0 must be set by a superuser

    - use `renice` to change the niceness value of an *existing* process
      ```
      [root@centos01 azureadmin]# renice -n 11 83883
      83883 (process ID) old priority 10, new priority 11
      ```

  - `ps -fax` can be used to view processes in a tree-like format (parent-child relationships)

  - Signals
    - Signals are like interrupts
    - sigstop and sigkill cannot be ignored by a process
      - sigkill immediately kills a process without allowing it to finish. This is commonly used when a process is no longer responding. 
    - You can view the signals using `kill -l`
      ```
      [root@centos01 azureadmin]# kill -l
       1) SIGHUP       2) SIGINT       3) SIGQUIT      4) SIGILL       5) SIGTRAP
       2) SIGABRT      7) SIGBUS       8) SIGFPE       9) SIGKILL     10) SIGUSR1
      1)  SIGSEGV     12) SIGUSR2     13) SIGPIPE     14) SIGALRM     15) SIGTERM
      2)  SIGSTKFLT   17) SIGCHLD     18) SIGCONT     19) SIGSTOP     20) SIGTSTP
      3)  SIGTTIN     22) SIGTTOU     23) SIGURG      24) SIGXCPU     25) SIGXFSZ
      4)  SIGVTALRM   27) SIGPROF     28) SIGWINCH    29) SIGIO       30) SIGPWR
      5)  SIGSYS      34) SIGRTMIN    35) SIGRTMIN+1  36) SIGRTMIN+2  37) SIGRTMIN+3
      6)  SIGRTMIN+4  39) SIGRTMIN+5  40) SIGRTMIN+6  41) SIGRTMIN+7  42) SIGRTMIN+8
      7)  SIGRTMIN+9  44) SIGRTMIN+10 45) SIGRTMIN+11 46) SIGRTMIN+12 47) SIGRTMIN+13
      8)  SIGRTMIN+14 49) SIGRTMIN+15 50) SIGRTMAX-14 51) SIGRTMAX-13 52) SIGRTMAX-12
      9)  SIGRTMAX-11 54) SIGRTMAX-10 55) SIGRTMAX-9  56) SIGRTMAX-8  57) SIGRTMAX-7
      10) SIGRTMAX-6  59) SIGRTMAX-5  60) SIGRTMAX-4  61) SIGRTMAX-3  62) SIGRTMAX-2
      11) SIGRTMAX-1  64) SIGRTMAX
      ```
    - Example: `kill -SIGHUP 1407` OR `kill -1 1407`
      - You can then see the result using systemctl
        ```
        [root@centos01 azureadmin]# systemctl status dovecot
        ● dovecot.service - Dovecot IMAP/POP3 email server
          Loaded: loaded (/usr/lib/systemd/system/dovecot.service; enabled; vendor preset: disabled)
          Active: active (running) since Tue 2022-08-30 14:18:41 UTC; 18h ago
            Docs: man:dovecot(1)
                  http://wiki2.dovecot.org/
          Process: 1369 ExecStartPre=/usr/libexec/dovecot/prestartscript (code=exited, status=0/SUCCESS)
        Main PID: 1407 (dovecot)
            Tasks: 3 (limit: 204310)
          Memory: 5.3M
          CGroup: /system.slice/dovecot.service
                  ├─ 1407 /usr/sbin/dovecot -F
                  ├─ 1461 dovecot/anvil
                  └─84378 dovecot/log

        Aug 30 14:18:41 centos01 systemd[1]: Starting Dovecot IMAP/POP3 email server...
        Aug 30 14:18:41 centos01 systemd[1]: Started Dovecot IMAP/POP3 email server.
        Aug 30 14:18:41 centos01 dovecot[1407]: master: Dovecot v2.3.8 (9df20d2db) starting up for imap (core dumps disabled)
        Aug 31 08:57:26 centos01 dovecot[1407]: master: Warning: SIGHUP received - reloading configuration
        ```
    - If you do not specify an operating with the `kill` command, SIGTERM will be used as a default
      - SIGTERM tells the process to gracefully shutdown

    - `pkill` can be used to kill process by name
      - To get a list of processes to kill `pgrep -a bash`
      - To then kill those processes: `pkill bash`

  - Background jobs
    - `CTRL + Z` sends a process to the background (doesn't work with all processes), using & with a command does the same. 
      - Example: `sleep 300 &`
    - `fg` can be used to bring a process that is in the background to the foreground
    - `jobs` can be used to view processes running in the background
    - Example:
      ```
      [root@centos01 azureadmin]# nano -z test
      Use "fg" to return to nano.

      [2]+  Stopped                 nano -z test
      [root@centos01 azureadmin]# jobs
      [1]-  Stopped                 man nano
      [2]+  Stopped                 nano -z test
      [root@centos01 azureadmin]# fg 2
      nano -z test
      ```
  - Viewing what files a process is using:
    - `lsof -p` can be used to see all files a process has open
      - Example:
        ```
        [root@centos01 azureadmin]# pgrep -a cron
        1386 /usr/sbin/crond -n
        [root@centos01 azureadmin]# lsof -p 1386
        COMMAND  PID USER   FD      TYPE             DEVICE  SIZE/OFF     NODE NAME
        crond   1386 root  cwd       DIR                8,2       270      128 /
        crond   1386 root  rtd       DIR                8,2       270      128 /
        crond   1386 root  txt       REG                8,2     80552 17384647 /usr/sbin/crond
        crond   1386 root  mem       REG                8,2     83760 17041984 /usr/lib64/libnss_files-2.28.so
        crond   1386 root  mem       REG                8,2   9253600 17676054 /var/lib/sss/mc/passwd
        crond   1386 root  mem       REG                8,2     46272 17675669 /usr/lib64/libnss_sss.so.2
        crond   1386 root  mem       REG                8,2 217796128 34698561 /usr/lib/locale/locale-archive
        crond   1386 root  mem       REG                8,2    321552 17041986 /usr/lib64/libpthread-2.28.so
        crond   1386 root  mem       REG                8,2     24928 17090995 /usr/lib64/libcap-ng.so.0.0.0
        crond   1386 root  mem       REG                8,2    543160 16923647 /usr/lib64/libpcre2-8.so.0.7.1
        crond   1386 root  mem       REG                8,2   3167872 16923700 /usr/lib64/libc-2.28.so
        crond   1386 root  mem       REG                8,2    123336 17091044 /usr/lib64/libaudit.so.1.0.0
        crond   1386 root  mem       REG                8,2     28856 16923702 /usr/lib64/libdl-2.28.so
        crond   1386 root  mem       REG                8,2     66520 17180400 /usr/lib64/libpam.so.0.84.2
        crond   1386 root  mem       REG                8,2    168536 16923653 /usr/lib64/libselinux.so.1
        crond   1386 root  mem       REG                8,2    278512 17846639 /usr/lib64/ld-2.28.so
        crond   1386 root    0r      CHR                1,3       0t0        7 /dev/null
        crond   1386 root    1u     unix 0xffff91c6ce0f2880       0t0    32976 type=STREAM
        crond   1386 root    2u     unix 0xffff91c6ce0f2880       0t0    32976 type=STREAM
        crond   1386 root    3u     unix 0xffff91c8fae28480       0t0    30261 type=DGRAM
        crond   1386 root    4r  a_inode               0,14         0    12289 inotify
        crond   1386 root    5u     unix 0xffff91c6d6c4b600       0t0  1657154 type=STREAM
        crond   1386 root    6r      REG                8,2   9253600 17676054 /var/lib/sss/mc/passwd
        ```
    - You can also point `lsof` at a file to see which processes are using it
      - Example:  
        ```
        [root@centos01 azureadmin]# lsof /bin/bash
        COMMAND   PID       USER  FD   TYPE DEVICE SIZE/OFF NODE NAME
        bash     6247 azureadmin txt    REG    8,2  1150584 1032 /usr/bin/bash
        bash    83356 azureadmin txt    REG    8,2  1150584 1032 /usr/bin/bash
        bash    83746 azureadmin txt    REG    8,2  1150584 1032 /usr/bin/bash
        bash    83788       root txt    REG    8,2  1150584 1032 /usr/bin/bash
        bash    83883       root txt    REG    8,2  1150584 1032 /usr/bin/bash
        ```

  - Locate and Analyze Log Files
    - Logging daemons collect and analyze log files
    - RSyslog is the most popular on Linux (Rocket-fast System for Logging)
      - Config file for rsyslogd is stored at `/etc/rsyslog.conf` and `/etc/rsyslog.d/`
    - logs are commonly stored in `/var/log`
    - You can search all logs using grep: `grep -r 'ssh' /var/log`
    - `journalctl` can be used to analyze log files
    - `journalctl -p` can be used to filter logs by priority
      ```
      [root@centos01 log]# journalctl -p 2
      -- Logs begin at Tue 2022-08-30 14:18:29 UTC, end at Wed 2022-08-31 09:19:25 UTC. --
      Aug 30 14:18:34 centos01 chronyd[748]: Fatal error : Could not open PHC
      ```

  - Scheduling Tasks
    - `cron`, `anacron`, and `at` can be used to schedule jobs
    - `cron` will not run a job if the system was powered off while the job was scheduled to run. `anacron` was created to resolve this shortcoming.
    - `at` will runs jobs only once
    - You can look at `/etc/crontab` to view an example cronjob (for syntax)
    - use `crontab -e` to edit the crontab of the user
    - use `crontab -l` to view the crontab of the user
    - use `crontab -r` to remove a crontab
    - Any files or executables in these directly will be run on a schedule according to the name of the directory (daily, hourly, etc.)
      ```
      /etc/cron.daily/   
      /etc/cron.hourly/  
      /etc/cron.monthly/ 
      /etc/cron.weekly/
      ```

    - edit the `/etc/anacron` tab to add an anacron job (the syntax for anacron is also in this file)
    - `anacron -T` can be used to verify the syntax in `/etc/anacron` is correct

    - `at` can be used to schedule a run-once command
      - Example: 
        ```
        [root@centos01 log]# at 'now + 3 hours'
        warning: commands will be executed using /bin/sh
        at> ls -lisa
        at> <EOT>
        job 3 at Wed Aug 31 12:58:00 2022
        [root@centos01 log]#
        ```
    - Use `CTRL + D` to save the `at` command
    - to verify cron jobs are executing, look at /var/log/cron
    - in /etc/crontab, the MAILTO line specifies who a status message is sent to every time a cronjob runs:
      ```
      [root@centos01 mail]# cat /etc/crontab
      SHELL=/bin/bash
      PATH=/sbin:/bin:/usr/sbin:/usr/bin
      MAILTO=root

      # For details see man 4 crontabs

      # Example of job definition:
      # .---------------- minute (0 - 59)
      # |  .------------- hour (0 - 23)
      # |  |  .---------- day of month (1 - 31)
      # |  |  |  .------- month (1 - 12) OR jan,feb,mar,apr ...
      # |  |  |  |  .---- day of week (0 - 6) (Sunday=0 or 7) OR sun,mon,tue,wed,thu,fri,sat
      # |  |  |  |  |
      # *  *  *  *  * user-name  command to be executed
      ```
      - check /var/spool/mail/<username> to view the messages 

  - Manage Software
    - use `dnf check-upgrade` on centos based systems to check for packages needing updates
    - use `dnf upgrade` to upgrade packages
    - if systemd or kernel components are upgraded, you will need to reboot the system
    - use `sudo dnf repolist` to view repositories
    - use `sudo dnf repolist --all` to view optional repos that can be enabled
    - use `sudo dnf config-manager --enable <repo name>` to enable a repo. Use the `--disable` flag to disable a repo
    - You can install groups of packages using `sudo dnf group install <group name>`
    - View groups of packages using `sudo dnf group list --hidden`
    - You can also install software using an RPM package
    - To find out which package installed a file: `dnf provides <file name>`
    - You can get a list of all the files a package contains: `dnf repoquery --list <package name>`
    - `/var/lib/rpm` contains metadata for installed packages
    - Example of a repo file:
      ```
      [code]
      name=Visual Studio Code
      baseurl=https://packages.microsoft.com/yumrepos/vscode
      enabled=1
      gpgcheck=1
      gpgkey=https://packages.microsoft.com/keys/microsoft.asc
      ```
    - Each repo file requires a unique ID, name, and base url
    - repo files are stored in `/etc/yum/yum.repos.d/`
    - Managing package groups
      - `dnf group list` can be used to view available and installed groups
        - environment groups are package groups containing entire runtime environments for a server, such as "Server with a GUI"
        - Groups are small groups of packages that serve a common purpose, such as "Container Management" or "Development Environment"

  - Verify the integrity and availability of resources
    - `df` aka 'disk free' can be used to view disk utilization
    - `du` can be used to view disk utilization for a directory
    - `free` can be used to view memory (RAM) stats
    - `lscpu` can be used to view info about the CPU
    - `lspci` can be used to view info about other hardware
    - `xfs_repair` can be used to repair xfs file systems
    - `fsck.ext4` can be used to repair EXT4 file systems
    - File systems must be unmounted to be repaired

  - Change Kernel Runtime Parameters
    - Kernel runtime params are just settings for how the kernel does its job
    - use `sysctl -a` to view kernel runtime params
    - use `sysctl -w <setting>` to modify a kernel parameter. This change is not persistent across reboots
    - create a .conf file in /etc/sysctl.d to make the change persistent across reboots. This file will not be recognized until the next reboot.
      If you want the change to be recognized immediately, you can use `sysctl -p <path to file>` 

  - SELinux and AppArmor File and Process Contexts
    - SELinux allows fine grained control of what actions are allowed or denied
    - `ls -Z` can be used to view SELinux labels for files
    - `ps axZ` can be used to view SELinux labels for processes

##### Commands
```
reboot
shutdown
systemctl

grub2-mkconfig 
grub2-install
lsblk

ps
top
htop
pgrep
pidof
nice
renice

kill
pkill

fg
jobs

lsof 

journalctl

cron
anacron
at

dnf
yum
rpm

df
du
free
uptime
lscpu
lsmem
lspci
lsblk
xfs_repair
fsck.ext4

sysctl
id
semanage
setenforce
chcon

grub2-mkconfig

```